[Markdown Guide](https://docs.gitlab.com/ee/user/markdown.html)

# Praxismodul I
## Thema 6: Process Organisation
## Hochschule Mainz
## Wirtschaftsinformatik dual B.Sc., Fachsemester 4

Durchgeführt von

| Student | Matrikel-Nummer |
|:-- |:--|
|Moritz Wetzel | 943566|
|Till Waller | 943565|
Fabian Harmsen| 943611|



## Gliederung

1. Projektziel
2. Wissenschaftliche Methoden
    2.1 Similar Activities
    2.2 Discover Roles
    2.3 Handover Of Work
3. Erklärungen zu den grafischen Ergebnissen
4. Configuration
5. Import
6. Similar Activities (Skript)
7. Discover Roles (Skript)
8. Handover Of Work (Skript)
9. Clustering Similar Activities
10. Clustering Working together
11. Fazit

## 1. Projektziel

Im Rahmen dieser Projektarbeit wird die Prozessorganisation eines zur Verfügung gestellten Datensatzes untersucht. Die uns zur Verfügung gestellte Logdatei stammt aus der BPI-Challenge 2017 und bestehen aus 2 Geschäftsprozessen. Unser Projektziel wird daraus bestehen, soziale Netzwerke innerhalb eines gewünschten Datensatzes zu entdecken. Hierfür werden die Daten mit Methoden der sozialen Netzwerkanalyse untersucht, um die sozialen Verbindungen im importierten Datensatz zu untersuchen und auszuwerten. Die Ergebnisse werden mithilfe eines Jupyter-Notebooks berechnet und visuell dargestellt werden.

## 2. Wissenschaftliche Methoden

In den folgenden Unterpunkten werden alle genutzen Methoden der Analyse sozialer Netzwerke theorisch dargestellt und erläutert. Als Grundlage für die theorische Recherche diente www.PM4PY.org.

//TODO: Recherche, Ausformulieren
### 2.1 Similar Activities

//TODO: Recherche, Ausformulieren
### 2.2 Discover Roles

//TODO: Recherche, Ausformulieren
### 2.3 Handover of Work



## 3. Erklärung zu den grafischen Ergebnissen

Im vorherigen Kapitel wurden alle in dieser Arbeit genutzten Methoden der Analyse von sozialen Netzwerken bereits erläutert. Bei der Generierung der Dateien zur Visualisierung der Ergebnisse werden webbasierte grafische Ausgaben erzeugt. Die Ausgaben setzen sich aus Nodes (Knoten) und Edges (Verbindungen) zusammen. Ein Node stellt im Rahmen unserer Arbeit immer einen Nutzer dar. Nach dem Clustering stellt ein Node hingegen eine Benutzergruppe dar. Die Benutzer beziehungsweise Benutzergruppen sind unterschiedlich stark über Edges miteinander verbunden. Die Stärke dieser Verbindung wird über Analysen festgestellt. Die damit erstellte Physiksimulation dient der visuellen Darstellung der Methodenergebnisse. Die Edges wirken wie Gummibänder, die die Nodes zusammenhalten. Die Nodes hingegen versuchen möglichst viel Abstand zu anderen Nodes zu haben. Dies führt dazu, dass Nodes, die aufgrund der Analysen mit starken Edges verbunden sind, sich annähern und weniger stark verbundene Nodes sich voneinander entfernen. Sobald sich die Physiksimulation beruhigt hat, kann man Gruppen von Nodes erkennen sowie die Verbindungen zu anderen Gruppen.

Die Analyseergebnisse können als HTML-Datei im Browser geöffnet werden und sind unter "result/*" zu finden.

*Wir erzeugen beim beziehungsweise nach dem Clustering keine grafische Ausgabe, da dies bei den getesteten Datensätzen keine neuen Erkenntnisse gebracht hat.*

## 4. Configuration

Die folgende Tabelle erklärt die Variablen, die im nächsten Code-Block definiert werden müssen. Die Logdatei muss, um richtig analysiert werden zu können, im XES-Dateiformat vorliegen und importiert werden.

| variable | type | description| default|
|:--|:--|:--|:--|
|log_path | (path)| path to the XES File|`data/test.xes`|
|use_networkx | (bool)| output via NetworkX or Pyvis| `False`|
|filter_timeframe | (bool)| filter the timeframe of the traces| `False` |
|filter_timeframe_intersecting | (bool)| use intersecting or contained| `False` |
|filter_timeframe_start | (dateTime)| start of the Timeframe| `1970-01-01 00:00:00` |
|filter_timeframe_end | (dateTime)| end of the Timeframe| `2038-12-31 00:00:00` |
|filter_performance| (bool)| filter on Traceperformance| `False` |
|filter_performance_min| (int)| minimum seconds of trace actions|`0` |
|filter_performance_max |(int)| maximum seconds of trace actions|`360000` |

Die Option `Pyvis` legt die Ergebnisse im Ordner `./result` ab, welcher schreibend Zugriff benötigt.

In [41]:
log_path = 'data/BPI_Challenge_2017_Offer_log.xes'

use_networkx = False

filter_timeframe = False
filter_timeframe_intersecting  = False
filter_timeframe_start = "2011-03-09 00:00:00"
filter_timeframe_end = "2011-03-09 00:00:00"

filter_performance = False
filter_performance_min = 0
filter_performance_max = 864000

## 5. Import
Im folgenden Codeblock werden alle Bibliotheken geladen die im laufenden Skript benötigt werden. Wenn die Bibliotheken nicht gefunden werden können, können sie über den Packagemanager pip seperat nachinstalliert werden.

**Unvollständige Liste der Pythonpackete (pip)**
 - pm4py
 - shutil
 - numpy
 - sklearn
 - scipy


In [42]:
import shutil
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.organizational_mining.sna import algorithm as sna
from pm4py.visualization.sna import visualizer as sna_visualizer
from pm4py.algo.organizational_mining.roles import algorithm as roles_discovery
from pm4py.algo.organizational_mining.sna import util
from pm4py.algo.filtering.log.timestamp import timestamp_filter
from pm4py.algo.filtering.log.cases import case_filter
from pm4py.algo.organizational_mining.local_diagnostics import algorithm as local_diagnostics
from pathlib import Path

Der Ordner für die Exportergebnisse (result) wird in diesem Codeblock erstellt.

In [43]:
if not use_networkx:
    Path("./result").mkdir(parents=True, exist_ok=True)

Dieser Code-Block lädt den in 4. importierten Datensatz und wendet die eingestellten Filter an.

In [44]:
log = xes_importer.apply(log_path)

if filter_timeframe:
    if not filter_timeframe_intersecting:
        log = timestamp_filter.filter_traces_contained(   log, filter_timeframe_start, filter_timeframe_end)
    else:
        log = timestamp_filter.filter_traces_intersecting(log, filter_timeframe_start, filter_timeframe_end)

if filter_performance:
    log = case_filter.filter_case_performance(log, filter_performance_min, filter_performance_max)

parsing log, completed traces ::   0%|          | 0/42995 [00:00<?, ?it/s]

## 6. Similar Activities (Skript)

In diesem Abschnitt wird die Methode "Similar Activities" auf den zuvor importierten Datensatz angewendet, um die Ähnlichkeit der Aktivitäten der verschiedenen Nutzer (Nodes) zu untersuchen. Die Theorie hinter der hier angewandten Methode kann man unter 2.1 lesen.


In [45]:
sa_metric  = sna.apply(log, variant=sna.Variants.JOINTACTIVITIES_LOG)

if use_networkx:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_ja_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_ja_py, "result/similar_activities.html")

## 7. Discover Roles (Skript)

In diesem Abschnitt wird die Methode “Discover Roles” auf den zuvor importierten Datensatz angewendet, um die Rollen eines Nutzers (Node) anhand seiner durchgeführten Aktivitäten zu erkennen. Dabei werden Nutzer mit den gleichen Aktivitäten gruppiert. Die Theorie hinter der hier angewandten Methode kann man unter 2.2 lesen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Rollenname auf Basis der Aktivitäten
2. Benutzer mit der Angabe wie aktiv der Nutzer in der Rolle war
3. `Leerzeile`

In [46]:
roles = roles_discovery.apply(log)

for x in roles:
    print(x[0])
    print(x[1])
    print("")

['O_Accepted']
{'User_95': 438, 'User_29': 1710, 'User_90': 1147, 'User_115': 744, 'User_87': 749, 'User_119': 333, 'User_99': 616, 'User_117': 306, 'User_30': 970, 'User_109': 1235, 'User_114': 64, 'User_113': 352, 'User_75': 1036, 'User_112': 362, 'User_118': 766, 'User_68': 971, 'User_116': 191, 'User_102': 943, 'User_120': 157, 'User_127': 105, 'User_121': 220, 'User_125': 178, 'User_83': 561, 'User_2': 1, 'User_128': 219, 'User_107': 424, 'User_122': 95, 'User_126': 43, 'User_27': 181, 'User_123': 214, 'User_129': 367, 'User_100': 914, 'User_133': 115, 'User_93': 138, 'User_31': 181, 'User_131': 66, 'User_134': 72, 'User_53': 18, 'User_106': 25, 'User_137': 1}

['O_Cancelled']
{'User_17': 112, 'User_1': 9982, 'User_29': 603, 'User_19': 67, 'User_38': 75, 'User_27': 76, 'User_3': 203, 'User_8': 10, 'User_87': 244, 'User_119': 150, 'User_20': 35, 'User_95': 143, 'User_112': 138, 'User_2': 158, 'User_89': 1, 'User_115': 177, 'User_9': 16, 'User_75': 375, 'User_113': 162, 'User_30': 3

## 8. Handover Of Work (Skript)

In diesem Abschnitt wird die Methode “Handover Of Work” auf den zuvor importierten Datensatz angewendet, um Nutzer herauszufinden, welche häufig miteinander arbeiten. Die Theorie hinter der hier angewandten Methode kann man unter 2.3 lesen.


In [47]:
hw_values = sna.apply(log, variant=sna.Variants.HANDOVER_LOG)

if use_networkx:
    gviz_hw_py = sna_visualizer.apply(hw_values, variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_hw_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_hw_py = sna_visualizer.apply(hw_values , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_hw_py, "result/handover_of_work.html")

## 9. Clustering Similar Activities

In diesem Abschnitt werden die unter 6 analysierten Nutzer in Nutzergruppen zusammengefasst, welche gleiche Aufgaben verrichtet haben. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [48]:
clustering_sa = util.cluster_affinity_propagation(sa_metric)

for group in clustering_sa:
    print("Gruppe: " + group)
    print(clustering_sa[group])
    print("")


Gruppe: -1
['User_1', 'User_10', 'User_100', 'User_101', 'User_102', 'User_103', 'User_104', 'User_105', 'User_106', 'User_107', 'User_108', 'User_109', 'User_11', 'User_110', 'User_111', 'User_112', 'User_113', 'User_114', 'User_115', 'User_116', 'User_117', 'User_118', 'User_119', 'User_12', 'User_120', 'User_121', 'User_122', 'User_123', 'User_124', 'User_125', 'User_126', 'User_127', 'User_128', 'User_129', 'User_13', 'User_130', 'User_131', 'User_132', 'User_133', 'User_134', 'User_135', 'User_136', 'User_137', 'User_138', 'User_139', 'User_14', 'User_140', 'User_141', 'User_142', 'User_143', 'User_144', 'User_15', 'User_16', 'User_17', 'User_18', 'User_19', 'User_2', 'User_20', 'User_21', 'User_22', 'User_23', 'User_24', 'User_25', 'User_26', 'User_27', 'User_28', 'User_29', 'User_3', 'User_30', 'User_31', 'User_32', 'User_33', 'User_34', 'User_35', 'User_36', 'User_37', 'User_38', 'User_39', 'User_4', 'User_40', 'User_41', 'User_42', 'User_43', 'User_44', 'User_45', 'User_46', '

/Users/moritzwetzel/PycharmProjects/PraxisModul1_PM4PY/venv/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(
/Users/moritzwetzel/PycharmProjects/PraxisModul1_PM4PY/venv/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:528: ConvergenceWarning: This model does not have any cluster centers because affinity propagation did not converge. Labeling every sample as '-1'.
  warnings.warn(


## 10. Clustering Handover Of Work

In diesem Abschnitt werden die unter 8 analysierten Nutzer als Gruppe zusammengefasst, welche häufig miteinander arbeiten. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [49]:
clustering_hw = util.cluster_affinity_propagation(hw_values)
for group in clustering_hw:
    print("Gruppe: " + group)
    print(clustering_hw[group])
    print("")

Gruppe: 4
['User_1', 'User_100', 'User_101', 'User_102', 'User_103', 'User_104', 'User_105', 'User_106', 'User_107', 'User_108', 'User_109', 'User_110', 'User_111', 'User_114', 'User_115', 'User_124', 'User_125', 'User_127', 'User_128', 'User_129', 'User_13', 'User_130', 'User_131', 'User_132', 'User_133', 'User_134', 'User_135', 'User_136', 'User_137', 'User_138', 'User_139', 'User_140', 'User_141', 'User_142', 'User_143', 'User_144', 'User_20', 'User_22', 'User_23', 'User_25', 'User_26', 'User_29', 'User_30', 'User_31', 'User_33', 'User_34', 'User_43', 'User_45', 'User_47', 'User_50', 'User_54', 'User_55', 'User_58', 'User_59', 'User_6', 'User_62', 'User_63', 'User_64', 'User_65', 'User_66', 'User_68', 'User_69', 'User_70', 'User_72', 'User_73', 'User_74', 'User_75', 'User_76', 'User_77', 'User_79', 'User_8', 'User_80', 'User_81', 'User_82', 'User_83', 'User_84', 'User_85', 'User_86', 'User_87', 'User_88', 'User_89', 'User_9', 'User_90', 'User_91', 'User_92', 'User_93', 'User_94', 'U

In [50]:
# this applies the organizational mining from an attribute that is in each event, describing the group that is performing the task.
ld = local_diagnostics.apply_from_group_attribute(log)
# GROUP RELATIVE FOCUS (on a given type of work) specifies how much a resource group performed this type of work
# compared to the overall workload of the group. It can be used to measure how the workload of a resource group
# is distributed over different types of work, i.e., work diversification of the group.
print("\ngroup_relative_focus")
print(ld["group_relative_focus"])
# GROUP RELATIVE STAKE (in a given type of work) specifies how much this type of work was performed by a certain
# resource group among all groups. It can be used to measure how the workload devoted to a certain type of work is
# distributed over resource groups in an organizational model, i.e., work participation by different groups.
print("\ngroup_relative_stake")
print(ld["group_relative_stake"])
# GROUP COVERAGE with respect to a given type of work specifies the proportion of members of a resource group that
# performed this type of work.
print("\ngroup_coverage")
print(ld["group_coverage"])
# GROUP MEMBER CONTRIBUTION of a member of a resource group with respect to the given type of work specifies how
# much of this type of work by the group was performed by the member. It can be used to measure how the workload
# of the entire group devoted to a certain type of work is distributed over the group members.
print("\ngroup_member_contribution")
print(ld["group_member_contribution"])



group_relative_focus
{}

group_relative_stake
{}

group_coverage
{}

group_member_contribution
{}
